In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error, precision_score
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier

In [12]:
# load the data
root = os.path.dirname(os.getcwd())
data_dir = os.path.join(root, "data")
files = {
    "long": {
        "lymphoma": [file for file in os.listdir(data_dir) if ("lungc" in file or "lymph" in file) and "long_cleaned" in file],
        "oncology": [file for file in os.listdir(data_dir) if ("lungc" not in file and "lymph" not in file) and "long_cleaned" in file]
    },
    "clinicalai": {
        "lymphoma": [file for file in os.listdir(data_dir) if ("lungc" in file or "lymph" in file) and "long_cleaned" not in file and "csv" in file],
        "oncology": [file for file in os.listdir(data_dir) if ("lungc" not in file and "lymph" not in file) and "long_cleaned" not in file and "cleaned_data_phase1" in file and "csv" in file]
    }
    }
files

{'long': {'lymphoma': ['long_cleaned_data_phase1_lungc_2bin_test.csv',
   'long_cleaned_data_phase1_lymph_2bin_train.csv'],
  'oncology': ['long_cleaned_data_phase1_oncol_2bin_test.csv',
   'long_cleaned_data_phase1_oncol_2bin_train.csv']},
 'clinicalai': {'lymphoma': ['cleaned_data_phase1_lymph_2bin_train.csv',
   'cleaned_data_phase1_lungc_2bin_test.csv'],
  'oncology': ['cleaned_data_phase1_2bin_test.csv',
   'cleaned_data_phase1_2bin_train.csv']}}

# Long model on Phase 1 Lymphoma data

In [13]:
# load the phase 1 lymphoma data (processed with long_processing.py)
train_data = pd.read_csv(os.path.join(root,"data",files["long"]["lymphoma"][1]))
train_data

,study_duration_days,study_eq_bins,study_eq_labels,num_oms,number_of_intervention_types,num_locations,number_of_conditions,allocation,masking,enroll_count,...,spacy_aes_outcome,spacy_osr_outcome,spacy_mtd_outcome,spacy_dor_outcome,spacy_dlt_outcome,spacy_cmax_outcome,spacy_aes_lead_outcome,spacy_dlt_lead_outcome,spacy_mtd_lead_outcome,spacy_tmax_outcome
0,901.0,Low,0,3,1,1,1,False,False,13.0,...,True,False,False,False,False,False,True,False,False,missing
1,912.0,Low,0,4,1,1,2,False,False,18.0,...,False,True,False,False,True,False,False,True,False,missing
2,1065.0,Low,0,5,1,3,2,missing,False,37.0,...,False,False,False,False,False,True,False,False,False,True
3,1916.0,High,1,4,1,2,2,missing,False,60.0,...,False,True,True,False,True,True,False,True,True,True
4,730.0,Low,0,1,1,3,2,False,False,43.0,...,False,False,False,False,False,False,False,False,False,missing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1047,3440.0,High,1,6,1,3,16,False,False,43.0,...,False,False,False,False,False,False,False,False,False,missing
1048,1646.0,Low,0,2,5,1,12,True,True,7.0,...,False,True,True,False,True,False,False,True,True,missing
1049,1461.0,Low,0,0,2,1,1,missing,True,30.0,...,False,False,False,False,False,False,False,False,False,missing
1050,2343.0,High,1,5,3,1,67,missing,False,43.0,...,False,False,True,False,True,False,False,True,True,missing


In [ ]:
# train the model


In [ ]:
# grid search

# retrain the model

In [ ]:
# model.predict on test set
# calculate metrics


# Long model on Phase 1 Oncology data

In [ ]:
# load the phase 1 oncology data (processed with long_processing.py)

In [ ]:
# train the model
# model.predict on test set
# calculate metrics

# ClinicalAI model on Phase 1 Lymphoma data

In [ ]:
# load the phase 1 lymphoma data (processed with processing.py)

In [ ]:
# train the model
# model.predict on test set
# calculate metrics

# ClinicalAI model on Phase 1 Oncology data

In [ ]:
# load the phase 1 oncology data (processed with processing.py)

In [ ]:
# train the model
# model.predict on test set
# calculate metrics

# Visualization

In [ ]:
# comparison table of metrics

In [ ]:
# venn diagram of feature comparison